1.setup the mario(gym_super_mario_bros==7.4.0)

pip install gym_super_mario_bros nes_py

In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
# env = JoypadSpace(env, SIMPLE_MOVEMENT)

# done = True
# for step in range(5000):
#     if done:
#         state = env.reset()
#     state, reward, done, info = env.step(env.action_space.sample())
#     env.render()

# env.close()

2.preprocess the environment(stable_baselines3==1.7.0)

pip install stable_baselines3[extra]==1.7.0

In [2]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import FrameStack, GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
# Import matplotlib
from matplotlib import pyplot as plt
Monitor_dir = './monitor/'
# 1.create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2.simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3.Monitor the environment to visualise the reward
env = Monitor(env, Monitor_dir)
# 4. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 5. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 6. Stack 4 frames to one state
env = VecFrameStack(env, 4, channels_order='last')

In [3]:
# state = env.reset()
# state.shape

In [4]:
# state, reward, done, info = env.step([env.action_space.sample()])

In [5]:
# To show four frames image
# plt.figure(figsize=(10,8))
# for idx in range(state.shape[3]):
#     plt.subplot(1,4,idx+1)
#     plt.imshow(state[0][:,:,idx])
# plt.show()

3. Train the RL model to play mario

In [ ]:
# Import os for file path management
import os
import numpy as np
# Import ppo algorithms
from stable_baselines3 import PPO
# Import BaseCallback for saving models
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.monitor import Monitor

class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq, log_dir, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print("Num timesteps: {}".format(self.num_timesteps))
                    print(
                        "Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(
                            self.best_mean_reward, mean_reward
                        )
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print("Saving new best model at {} timesteps".format(x[-1]))
                        print("Saving new best model to {}.zip".format(self.save_path))
                    self.model.save(self.save_path)

        return True

class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

# Setup model saving callback
# I wanna show the change of maorio's learning so I choose to save the model every check_freq.
# Actually you can use the SaveOnBestTrainingRewardCallback
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR, verbose=1)

#PPO to train the model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512)
model.learn(total_timesteps=5000000, callback=callback)